In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0


In [ ]:
idx = np.argsort(y_train)
x_train = x_train[idx]
y_train = y_train[idx]

In [ ]:
idx1= np.argsort(y_test)
x_test = x_test[idx1]
y_test = y_test[idx1]

In [ ]:
count=0
list1=[]
for j in range(10):
  list1.append(count)
  for i in y_train:
    if i == j:
      count = count+1
  print(count)

In [ ]:
count=0
list2=[]
for j in range(10):
  list2.append(count)
  for i in y_test:
    if i == j:
      count = count+1
  print(count)

In [ ]:
list1.append(60000)
list2.append(10000)

In [ ]:
iid_data_x = []
iid_data_y = []
for i in range(10):
  iid_data_x.append(x_train[list1[i]:list1[i+1]])
  iid_data_y.append(y_train[list1[i]:list1[i+1]])

In [ ]:
import random

mixin_ratio = 0.5
clients = []
y_train = []
for i in range(10):
  empty = np.zeros((6000,28,28,1))
  num_samp = int(6000*mixin_ratio)
  samp_1 = i
  samp_2 = (i+1)%10
  y_temp= np.zeros((6000))
  y_temp[:num_samp] = samp_1
  y_temp[num_samp:] = samp_2
  y_temp = list(y_temp)
  
  y_train = y_train + y_temp
  
  empty[:num_samp] = iid_data_x[samp_1][:(num_samp)]
  empty[num_samp:] = iid_data_x[samp_2][:(6000-num_samp)]

  clients.append(empty)

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
x_train_list = []
history_list = []
autoencoder_list = []
encoded_images_train = []
decoded_images_train = []
encoded_images_test = []
decoded_images_test = []

for i in range(10):

  latent_dim = 2

  encoder_inputs = keras.Input(shape=(28, 28, 1))
  x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
  x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Flatten()(x)
  x = layers.Dense(16, activation="relu")(x)
  z_mean = layers.Dense(latent_dim, name="z_mean")(x)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
  z = Sampling()([z_mean, z_log_var])
  encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
  encoder.summary()

  latent_inputs = keras.Input(shape=(latent_dim,))
  x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
  x = layers.Reshape((7, 7, 64))(x)
  x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
  x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
  decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
  decoder.summary()

  autoencoder_list.append(VAE(encoder, decoder)) 
  autoencoder_list[i].compile(optimizer=tf.keras.optimizers.Adam())
  autoencoder_list[i].fit(iid_data_x[i], epochs=30, batch_size = 16)
  
  # n = 10
  # plt.figure(figsize=(20, 4))
  # for j in range(n):
  #   # display original
  #   ax = plt.subplot(2, n, j + 1)
  #   plt.imshow(iid_data_x[i][j,:,:,0])
  #   plt.title("original")
  #   plt.gray()
  #   ax.get_xaxis().set_visible(False)
  #   ax.get_yaxis().set_visible(False)

  #   # display reconstruction
  #   ax = plt.subplot(2, n, j + 1 + n)
  #   plt.imshow(encoded_images_train[i].reshape(encoded_images_train[i].shape[0],8,8,1)[j,:,:,0])
  #   plt.title("reconstructed")
  #   plt.gray()
  #   ax.get_xaxis().set_visible(False)
  #   ax.get_yaxis().set_visible(False)
  # plt.show()

In [ ]:
# # # (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
# # # mnist_digits = np.concatenate([x_train, x_test], axis=0)
# # # mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

# vae = VAE(encoder, decoder)
# vae.compile(optimizer=keras.optimizers.Adam())
# vae.fit(iid_data_x[2], epochs=2, batch_size=128)

In [ ]:
overall_encoded_train = []
digit_size = 28
scale = 1
# figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates corresponding to the 2D plot
# of digit classes in the latent space
for r in range(10):
  print(r)
  grid_x = np.linspace(-scale, scale, 70)
  grid_y = np.linspace(-scale, scale, 70)[::-1]
  for i, yi in enumerate(grid_y):
      for j, xi in enumerate(grid_x):
          z_sample = np.array([[xi, yi]])
          x_decoded = autoencoder_list[r].decoder.predict(z_sample).reshape(28,28,1)
          overall_encoded_train.append(x_decoded)
        # print(x_decoded.shape)
        # n = 10
        # plt.figure()
        # plt.imshow(x_decoded[:,:,0])
        # plt.title("original")
        # plt.gray()
        # plt.get_xaxis().set_visible(False)
        # plt.get_yaxis().set_visible(False)
        # plt.show()

        # digit = x_decoded[0].reshape(digit_size, digit_size)

In [ ]:
y_train_new = np.zeros((49000))
for i in range(10):
  for j in range(4900):
    y_train_new[i*4900+j] = i

In [ ]:
x_train_new = np.array(overall_encoded_train)

In [ ]:
x_train_new.shape

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(10, activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
from sklearn.utils import shuffle
overall_encoded_train, y_train = shuffle(x_train_new,y_train_new, random_state=7)

In [ ]:
y_train_new = tf.one_hot(y_train_new.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
history = model2.fit(x_train_new, y_train_new,
                    batch_size=16,
                    epochs=1)

In [ ]:
test_loss, test_acc = model2.evaluate(x_test, y_test)

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for j in range(n):
  # display original
  ax = plt.subplot(2, n, j + 1)
  plt.imshow(x_train_new[j+4900*8,:,:,0])
  plt.title("new")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # # display reconstruction
  # ax = plt.subplot(2, n, j + 1 + n)
  # plt.imshow(encoded_images_train[i].reshape(encoded_images_train[i].shape[0],8,8,1)[j,:,:,0])
  # plt.title("reconstructed")
  # plt.gray()
  # ax.get_xaxis().set_visible(False)
  # ax.get_yaxis().set_visible(False)
plt.show()